<a href="https://colab.research.google.com/github/rajasreekalli/Data-Visualization/blob/main/First_Modell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [134]:
import pandas as pd
import numpy as np

In [135]:
df = pd.read_csv('/content/drive/MyDrive/insurance.csv')
df

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [136]:
# import train test split from sklearn
from sklearn.model_selection import train_test_split

In [137]:
# 1. Define features (X) and target (y)
X = df.drop(columns=['charges'])
y = df['charges']

In [138]:
# Check missing values
print(df.isna().sum().sum(), 'missing values')

0 missing values


In [139]:
# 2. Train test split the data to prepare for machine learning
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [140]:
# 3. Identify each feature as numerical, ordinal, or nominal.
# Ans: In my point of view age, bmi, children are ordinal. May be these columns also consider numeric. Smoker, sex, region are nominal.

In [141]:
# 4. One Hot Encode the nominal features
# imports
from sklearn.compose import make_column_selector
from sklearn.impute import SimpleImputer

In [142]:
sex = {'male': 1, 'female': 0}
smoker = {'yes': 1, 'no': 0}
region = {'northwest': 0, 'southeast': 1, 'southwest': 2, 'northeast': 3}
X_train['sex'] = X_train['sex'].replace(sex)
X_train['smoker'] = X_train['smoker'].replace(smoker)
X_train['region'] = X_train['region'].replace(region)
X_train

,age,sex,bmi,children,smoker,region
693,24,1,23.655,0,0,0
1297,28,0,26.510,2,0,1
634,51,1,39.700,1,0,2
1022,47,1,36.080,1,1,1
178,46,0,28.900,2,0,2
...,...,...,...,...,...,...
1095,18,0,31.350,4,0,3
1130,39,0,23.870,5,0,1
1294,58,1,25.175,0,0,3
860,37,0,47.600,2,1,2


In [143]:
sex = {'male': 1, 'female': 0}
smoker = {'yes': 1, 'no': 0}
region = {'northwest': 0, 'southeast': 1, 'southwest': 2, 'northeast': 3}
X_test['sex'] = X_test['sex'].replace(sex)
X_test['smoker'] = X_test['smoker'].replace(smoker)
X_test['region'] = X_test['region'].replace(region)
X_test

,age,sex,bmi,children,smoker,region
764,45,0,25.175,2,0,3
887,36,0,30.020,0,0,0
890,64,0,26.885,0,1,0
1293,46,1,25.745,3,0,0
259,19,1,31.920,0,1,0
...,...,...,...,...,...,...
342,60,0,27.550,0,0,3
308,58,1,34.865,0,0,3
1128,34,1,32.800,1,0,2
503,19,1,30.250,0,1,1


In [144]:
#make categorical and numeric selectors
cat_selector = make_column_selector(dtype_include='object')

In [145]:
cat_data = X_train[cat_selector(X_train)]
cat_data


""
693
1297
634
1022
178
...
1095
1130
1294
860


In [146]:
from sklearn.preprocessing import OneHotEncoder
#instantiate one hot encoder
ohe_encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
ohe_encoder.fit(cat_data)
cat_ohe = ohe_encoder.transform(cat_data)
cat_ohe

array([], shape=(1003, 0), dtype=float64)

In [147]:
pd.DataFrame(cat_ohe, columns=ohe_encoder.get_feature_names(cat_data.columns))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


""
0
1
2
3
4
...
998
999
1000
1001


In [148]:
# 5. Instantiate a linear regression model
from sklearn.linear_model import LinearRegression

In [149]:
reg = LinearRegression()

In [150]:
X_train.shape

(1003, 6)

In [151]:
X_test.shape

(335, 6)

In [152]:
# 6. Fit the model on the training data
reg.fit(X_train,y_train)

LinearRegression()

In [153]:
# 7. Evaluate the model performance on both the training set and the test set using R^2 score

In [154]:
from sklearn.metrics import r2_score

In [155]:
train_score = reg.score(X_train, y_train)
print(train_score)

0.7442478537980548


In [156]:
test_score = reg.score(X_test, y_test)
print(test_score)

0.7653556681743363
